In [1]:
!pip install duckduckgo-search requests beautifulsoup4 gradio
# duckduckgo-search for live web search
# requests - HTTP library used to make web requests (fetch pages)
# beautifulsoup4 - HTML parser library (BeautifulSoup) to parse and extract text from HTML
# library to build web UI for ML demos (the chatbot interface)


# ----------------------------
# 📌 Import Required Libraries
# ----------------------------
import gradio as gr
from huggingface_hub import InferenceClient
import requests
from bs4 import BeautifulSoup #websites scrawling and scrapping
from duckduckgo_search import DDGS  # For live web search


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 40.5 MB/s eta 0:00:00


In [2]:
# ----------------------------
# 📌 Initialize LLaMA 3
# ----------------------------
client = InferenceClient(
    model="meta-llama/Meta-Llama-3-70B-Instruct",
    token=""  # Replace with your HF key
)

chat_history = []  # To remember past conversation

In [3]:
# ----------------------------
# 📌 Function: Fetch content from a given website URL
# ----------------------------
def fetch_website_info(url):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text(separator=' ', strip=True)
        return text[:1000]  # Limit to avoid overload
    except Exception as e:
        return f"Could not fetch website: {e}"

In [4]:
# ----------------------------
# 📌 Function: Perform live web search
# ----------------------------
def search_web(query):
    try:
        results = []
        with DDGS() as ddgs:
            for r in ddgs.text(query, max_results=2):
                results.append(f"{r['title']}: {r['href']}") # title :website , href : https://hkbccjnkbukcj.com
        return "\n".join(results) if results else "No live results found."
    except Exception as e:
        return f"Search error: {e}"

In [5]:
# ----------------------------
# 📌 Main Chatbot Function
# ----------------------------
def chat_with_ai(user_input):
    global chat_history

    if not user_input.strip():
        return "Please type something!"

    # Add system prompt once
    if not chat_history:
        chat_history.append({
            "role": "system",
            "content": "You are a smart, friendly assistant with access to live search and website reading."
        })

    # Agent Decision: If query has URL
    if "http" in user_input:
        for word in user_input.split():
            if word.startswith("http"):
                site_data = fetch_website_info(word)
                user_input += f"\n\n[Website Data Extracted: {site_data}]"

    # Agent Decision: If query needs fresh info (keywords like latest, today, now)
    elif any(keyword in user_input.lower() for keyword in ["latest", "today", "now", "current price", "news"]):
        search_results = search_web(user_input)
        user_input += f"\n\n[Live Search Results: {search_results}]"

    # Add to chat history
    chat_history.append({"role": "user", "content": user_input}) # addes all questions and answers in history

    try:
        resp = client.chat_completion(
            messages=chat_history,
            max_tokens=500,
            # min_tokens=100,
            temperature=0.7
        )
        assistant_msg = resp.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

    chat_history.append({"role": "assistant", "content": assistant_msg})
    return assistant_msg

In [6]:
# ----------------------------
# 📌 Gradio Interface
# ----------------------------
gr.Interface(
    fn=chat_with_ai,
    inputs=gr.Textbox(lines=2, placeholder="Ask anything… (e.g., 'latest price of iPhone 15' or paste a link)", label="Your Prompt"),
    outputs=gr.Textbox(label="AI Response", lines=10),
    title="🤖 Smart Chatbot (LLaMA3 + Live Search + Website Reader)",
    description="Chat with AI that can search the web and read websites for updated information."
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1b5868630ab7653001.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
